In [2]:
import json
import os
from dotenv import load_dotenv
from langchain.llms import AzureOpenAI
import openai
from IPython.display import display, HTML, JSON, Markdown

/Users/liorking/Projects/AOAI-workshop/.venv/lib/python3.9/site-packages/urllib3/__init__.py:34: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


In [3]:
load_dotenv()
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
OPENAI_DEPLOYMENT_ENDPOINT = os.getenv("OPENAI_DEPLOYMENT_ENDPOINT")
OPENAI_DEPLOYMENT_NAME = os.getenv("OPENAI_COMPLETION_DEPLOYMENT_NAME")
OPENAI_MODEL_NAME = os.getenv("OPENAI_MODEL_NAME")
OPENAI_DEPLOYMENT_VERSION = os.getenv("OPENAI_DEPLOYMENT_VERSION")

client = AzureOpenAI(
  azure_endpoint = OPENAI_DEPLOYMENT_ENDPOINT, 
  api_key=OPENAI_API_KEY,  
  api_version="2023-05-15"
)


In [4]:
def init_llm(model=OPENAI_MODEL_NAME,
             deployment_name=OPENAI_DEPLOYMENT_NAME,
             openai_api_version=OPENAI_DEPLOYMENT_VERSION,
             azure_endpoint=OPENAI_DEPLOYMENT_ENDPOINT,
             temperature=0,
             max_tokens=400,
             top_p=1,
             ):

    llm = AzureOpenAI(deployment_name=deployment_name,
                      model=model,
                      openai_api_version=openai_api_version,
                      azure_endpoint=azure_endpoint,
                      temperature=temperature,
                      max_tokens=max_tokens,
                      top_p=top_p,
                      model_kwargs={"stop": ["<|im_end|>"]}
                      )
    return llm
llm = init_llm()

##### Let's do some math

In [20]:
# OpenAI gets this wrong!!
prompt = f"""Determine if the student's solution delimited by the triple backticks is correct or not.
Question: When David was 3 years old, his sister was twice my age. Now his sister is 40 years old. How old is David? 
Student's answer: ```David is 37 years old.``` 
If student is correct, then write 'Correct', otherwise write 'Incorrect'.  
<|im_end|>
"""

sum = llm(prompt)
display(HTML(sum))

##### Ask the model to solve it and then to compare both solutions and conclude which is correct.
 Define the task as a list of sub-tasks and ask the model to perform them in a specific order (splitting the task into sub-tasks technique).   
 Then ask the model to compare its own solution with the solution provided by the model and conclude which one is correct.
 
 Ask the model to share its reasoning for the conclusion.
 This method is called **_Chain of Thought_**.

In [21]:
Question = f"""When David was 3 years old, his sister was twice my age. Now his sister is 40 years old. How old is David?"""
Student_Solution = f"""David is 37 years old."""

prompt = f""" Determine if the Student's Solution for the Question is correct or not.
To solve the problem do the following:
1 - First, work out your OWN solution to the problem. Evaluate your final result to make sure it is correct and adheres to the question's conditions. 
    Reason about every step of your solution and make sure it is correct. 
2 - Second, compare your solution to the student's solution and evaluate if the student's solution is correct or not.
Don't decide if the student's solution is correct until you have done the problem yourself.

Student's Solution: ```{Student_Solution}```
Question: ```{Question}```


Use the following output format:
Actual solution steps: <your own solution steps>
\nStudent's solution: <student's solution>
\nStudent's solution is correct: <true/false>
<|im_end|>

"""

llm = init_llm()
sum = llm(prompt)
display(Markdown(sum))

Actual solution steps:
1. Let x be the age of David's sister when David was 3 years old.
2. Since David's sister was twice my age, x = 2 * 3 = 6.
3. Now, David's sister is 40 years old, so x + 37 = 40.
4. Therefore, x = 40 - 37 = 3.
5. David is 37 years old.

Student's solution: David is 37 years old.

Student's solution is correct: True